<a href="https://colab.research.google.com/github/lpleme/Machine_Learning/blob/main/bikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import metrics
from keras import models  
from sklearn.metrics import r2_score

SHUFFLE_BUFFER = 500
#BATCH_SIZE = 2
#BATCH_SIZE = 5 # Increased the amount of nodes, then we increased the total number of epochs which show how many times it runs through the model.
#BATCH_SIZE = 2 # epoch 25 result = 57, result = 42.9661
#BATCH_SIZE = 2 # added season and weathersit not hot_encoded
#BATCH_SIZE = 2 # hot encoded season and weather sit
#BATCH_SIZE = 2

#This is the batch size we'll be using
BATCH_SIZE = 32


bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

In [ ]:
#Splits the date into month, day, and year
bikes["month"] = bikes.dteday.str.split('/').str[0]
bikes["day"] = bikes.dteday.str.split('/').str[1]
bikes["year"] = bikes.dteday.str.split('/').str[2]

In [ ]:
#Makes a rent column that adds the casual and registered users
bikes['rent'] = bikes['casual'] + bikes['registered']
bikes

bikes1 = bikes

bikes1 = pd.get_dummies(bikes1, columns = ['season', 'weathersit'])
#bikes1 = pd.get_dummies(bikes1, columns = ['season', 'weathersit','month','year','day'])
bikes1.head()

,dteday,hr,holiday,workingday,hum,windspeed,temp_c,feels_like_c,casual,registered,month,day,year,rent,season_1,season_2,season_3,season_4,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,1/1/11,0,0,0,0.81,0,3.28,3.0014,3,13,1,1,11,16,1,0,0,0,1,0,0,0
1,1/1/11,1,0,0,0.80,0,2.34,1.9982,8,32,1,1,11,40,1,0,0,0,1,0,0,0
2,1/1/11,2,0,0,0.80,0,2.34,1.9982,5,27,1,1,11,32,1,0,0,0,1,0,0,0
3,1/1/11,3,0,0,0.75,0,3.28,3.0014,3,10,1,1,11,13,1,0,0,0,1,0,0,0
4,1/1/11,4,0,0,0.75,0,3.28,3.0014,0,1,1,1,11,1,1,0,0,0,1,0,0,0


In [ ]:
#Sets the target
target = bikes['rent']
target = bikes1['rent']

In [ ]:
numeric_feature_names = ['hr', 'holiday', 'workingday',  'hum', 'windspeed', 'temp_c', 'feels_like_c', 'season', 'weathersit']
numeric_features = bikes[numeric_feature_names]
numeric_features.head()


#We will use these numeric features in our model. This turns out values into something that the model can read
numeric_feature_names1 = ['hr', 'holiday', 'workingday',  'hum', 'windspeed', 'temp_c', 'feels_like_c', 'season_1', 'season_2', 'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3', 'weathersit_4']
#numeric_feature_names1 = ['hr', 'holiday', 'workingday',  'hum', 'windspeed', 'temp_c', 'feels_like_c', 'season_1', 'season_2', 'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3', 'weathersit_4', 'month', 'day', 'year'] # I tried adding month, day, year but it said that it failed to convert those to numpy
numeric_features1 = bikes1[numeric_feature_names1]
numeric_features1.head()

,hr,holiday,workingday,hum,windspeed,temp_c,feels_like_c,season_1,season_2,season_3,season_4,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,0,0,0,0.81,0,3.28,3.0014,1,0,0,0,1,0,0,0
1,1,0,0,0.80,0,2.34,1.9982,1,0,0,0,1,0,0,0
2,2,0,0,0.80,0,2.34,1.9982,1,0,0,0,1,0,0,0
3,3,0,0,0.75,0,3.28,3.0014,1,0,0,0,1,0,0,0
4,4,0,0,0.75,0,3.28,3.0014,1,0,0,0,1,0,0,0


In [ ]:
#Converting our numeric features readable by tensor
#tf.convert_to_tensor(numeric_features)
tf.convert_to_tensor(numeric_features1)

<tf.Tensor: shape=(16637, 15), dtype=float64, numpy=
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [21.,  0.,  1., ...,  0.,  0.,  0.],
       [22.,  0.,  1., ...,  0.,  0.,  0.],
       [23.,  0.,  1., ...,  1.,  0.,  0.]])>

In [ ]:
#normalizer = tf.keras.layers.Normalization(axis=-1)
#normalizer.adapt(numeric_features)

#Normalizes our values so we don't have major outliers while running the model
normalizer1 = tf.keras.layers.Normalization(axis=-1)
normalizer1.adapt(numeric_features1)

In [ ]:
#normalizer(numeric_features.iloc[:3])
# shows what is going on


normalizer1(numeric_features1.iloc[:3])
# shows what is going on

<tf.Tensor: shape=(3, 15), dtype=float32, numpy=
array([[-1.6702179 , -0.17180604, -1.4725312 ,  0.9587179 , -1.5609912 ,
        -1.3762757 , -1.1349833 ,  1.7832977 , -0.60047174, -0.60853565,
        -0.53962135,  0.7083815 , -0.5817692 , -0.2970507 , -0.01342958],
       [-1.5255835 , -0.17180604, -1.4725312 ,  0.90696377, -1.5609912 ,
        -1.4802456 , -1.2235863 ,  1.7832977 , -0.60047174, -0.60853565,
        -0.53962135,  0.7083815 , -0.5817692 , -0.2970507 , -0.01342958],
       [-1.3809493 , -0.17180604, -1.4725312 ,  0.90696377, -1.5609912 ,
        -1.4802456 , -1.2235863 ,  1.7832977 , -0.60047174, -0.60853565,
        -0.53962135,  0.7083815 , -0.5817692 , -0.2970507 , -0.01342958]],
      dtype=float32)>

In [ ]:
#This it the function that creates the model and its parameters. 
# KEEP THIS TO LEARN FROM THIS #2
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer1,
    tf.keras.layers.Dense(30, activation='relu'), #the more perceptrons/nodes, the more of the abstract features () you can take. Just like from the interactive video, I'm adding 10 neurons to train off. 15 here instead.
    tf.keras.layers.Dense(30, activation='relu'), #this is the other hidden layer, combines the inputs from the previous layer
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(30, activation='relu'), # I changed the dense thing from 10 to 15 to account for the 15 features I'm implementing. I then changed it from 15 to 20 and got a result of 32.3014
    #tf.keras.layers.Dense(20, activation='relu'), # I tried adding another hidden layer but I ended up only getting 33.5268
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                #(this is for a classification )loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                loss = tf.keras.losses.MeanAbsoluteError(), # this one is for regressions
                metrics = ['mean_absolute_error'])
                # metrics = ['accuracy'] # This one for classification


  # accuracy is more for classification
  # for regression we would want to switch to mean^2; r^2
  # I can add more tf.keras.layers.Dense intomy model
  return model

In [ ]:
#2
#This runs the model and trains the neural network
model = get_basic_model()
model.fit(numeric_features1, target, epochs=140, batch_size=BATCH_SIZE)

Epoch 1/140
520/520 [==============================] - 2s 2ms/step - loss: 112.3184 - mean_absolute_error: 112.3184
Epoch 2/140
520/520 [==============================] - 1s 2ms/step - loss: 86.3781 - mean_absolute_error: 86.3781
Epoch 3/140
520/520 [==============================] - 1s 2ms/step - loss: 74.1865 - mean_absolute_error: 74.1865
Epoch 4/140
520/520 [==============================] - 1s 2ms/step - loss: 70.3007 - mean_absolute_error: 70.3007
Epoch 5/140
520/520 [==============================] - 1s 2ms/step - loss: 68.0572 - mean_absolute_error: 68.0572
Epoch 6/140
520/520 [==============================] - 1s 2ms/step - loss: 66.3615 - mean_absolute_error: 66.3615
Epoch 7/140
520/520 [==============================] - 1s 2ms/step - loss: 64.3472 - mean_absolute_error: 64.3472
Epoch 8/140
520/520 [==============================] - 1s 2ms/step - loss: 62.0678 - mean_absolute_error: 62.0678
Epoch 9/140
520/520 [==============================] - 1s 2ms/step - loss: 59.3927 - m

In [ ]:
#2
numeric_dataset = tf.data.Dataset.from_tensor_slices((numeric_features1, target))

for row in numeric_dataset.take(3):
  print(row)

(<tf.Tensor: shape=(15,), dtype=float64, numpy=
array([0.    , 0.    , 0.    , 0.81  , 0.    , 3.28  , 3.0014, 1.    ,
       0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ])>, <tf.Tensor: shape=(), dtype=int64, numpy=16>)
(<tf.Tensor: shape=(15,), dtype=float64, numpy=
array([1.    , 0.    , 0.    , 0.8   , 0.    , 2.34  , 1.9982, 1.    ,
       0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ])>, <tf.Tensor: shape=(), dtype=int64, numpy=40>)
(<tf.Tensor: shape=(15,), dtype=float64, numpy=
array([2.    , 0.    , 0.    , 0.8   , 0.    , 2.34  , 1.9982, 1.    ,
       0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ])>, <tf.Tensor: shape=(), dtype=int64, numpy=32>)


In [ ]:
#This will shuffel our numeric dataset and run the model again
numeric_batches = numeric_dataset.shuffle(1000).batch(BATCH_SIZE)

model = get_basic_model()
model.fit(numeric_batches, epochs=140)

Epoch 1/140
520/520 [==============================] - 1s 2ms/step - loss: 100.1323 - mean_absolute_error: 100.1323
Epoch 2/140
520/520 [==============================] - 1s 2ms/step - loss: 85.4510 - mean_absolute_error: 85.4510
Epoch 3/140
520/520 [==============================] - 1s 2ms/step - loss: 72.1141 - mean_absolute_error: 72.1141
Epoch 4/140
520/520 [==============================] - 1s 2ms/step - loss: 64.1916 - mean_absolute_error: 64.1916
Epoch 5/140
520/520 [==============================] - 1s 2ms/step - loss: 60.5782 - mean_absolute_error: 60.5782
Epoch 6/140
520/520 [==============================] - 1s 2ms/step - loss: 57.9402 - mean_absolute_error: 57.9402
Epoch 7/140
520/520 [==============================] - 1s 2ms/step - loss: 55.1550 - mean_absolute_error: 55.1550
Epoch 8/140
520/520 [==============================] - 1s 2ms/step - loss: 52.0237 - mean_absolute_error: 52.0237
Epoch 9/140
520/520 [==============================] - 1s 2ms/step - loss: 47.8446 - m